# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [16]:
# Jupyter Notebook URL http://localhost:8888/notebooks/WeatherPy/VacationPy.ipynb

# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [17]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nova sintra,14.8667,-24.7167,25.50,77,45,3.78,CV,1723137279
1,1,talnakh,69.4865,88.3972,11.62,61,5,4.84,RU,1723137280
2,2,bredasdorp,-34.5322,20.0403,11.71,79,23,3.51,ZA,1723137282
3,3,papatowai,-46.5619,169.4708,6.88,89,99,1.37,NZ,1723137283
4,4,katsuren-haebaru,26.3369,127.8719,28.43,90,0,2.06,JP,1723137284


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    color='City', 
    size='Humidity', 
    tiles='OSM', 
    frame_width=800, 
    frame_height=600,
    hover_cols=['City', 'Humidity']
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
152,152,hasaki,35.7333,140.8333,26.97,89,0,2.74,JP,1723137453
162,162,fort mcmurray,56.7268,-111.3810,21.73,52,0,2.06,CA,1723137429
211,211,berlin,52.5244,13.4105,23.86,52,0,3.60,DE,1723136940
226,226,vila franca do campo,37.7167,-25.4333,25.63,60,0,2.13,PT,1723137535
252,252,yenifakili,39.2114,35.0004,25.66,21,0,2.83,TR,1723137564


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,nova sintra,CV,14.8667,-24.7167,77,
1,talnakh,RU,69.4865,88.3972,61,
2,bredasdorp,ZA,-34.5322,20.0403,79,
3,papatowai,NZ,-46.5619,169.4708,89,
4,katsuren-haebaru,JP,26.3369,127.8719,90,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nova sintra - nearest hotel: Residência Ka Dencho
talnakh - nearest hotel: Талнах
bredasdorp - nearest hotel: Victoria Hotel
papatowai - nearest hotel: No hotel found
katsuren-haebaru - nearest hotel: No hotel found
patterson - nearest hotel: Hampton Inn & Suites
rongelap - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
tiksi - nearest hotel: Арктика
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
kimbe - nearest hotel: Kimbe Bay Hotel
bengkulu - nearest hotel: Dena Hotel
encheng - nearest hotel: 金泉酒店
aasiaat - nearest hotel: SØMA
arraial do cabo - nearest hotel: No hotel found
bonthe - nearest hotel: No hotel found
west island - nearest hotel: Cocos Village Bungalows
howard springs - nearest hotel: Rydges Palmerston
mount isa - nearest hotel: Ibis Styles
puerto natales - nearest hotel: Dorotea Patagonia Hostel
cabo san lucas - nearest hotel: Comfort Rooms
usogorsk - nearest hotel: Единственная
grytviken - nearest hotel: No hotel foun

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    color='Humidity',  # Use the 'Humidity' column for coloring
    cmap='viridis',  # Colormap for numerical values
    size='Humidity', 
    tiles='OSM', 
    frame_width=800, 
    frame_height=600,
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity']  # Include Hotel Name and Country in hover
)

# Display the map
map_plot